In [1]:
import setup

setup.init_django()

In [2]:
from rag import (
    db as rag_db,
    engines as rag_engines,
    settings as rag_settings,
    updaters as rag_updaters,
    sync as rag_sync, 
    patches as rag_patches,
)


In [3]:
from typing import Optional, Union
from sqlalchemy import create_engine, text

In [4]:
import logging
logging.basicConfig(level=logging.INFO)

rag_settings.init()
rag_db.init_vector_db()
rag_sync.full_sync()

Starting sync for 0 documents across all models
Full synchronization completed


In [5]:
employee_index = rag_engines.get_model_index("Employee")
product_index = rag_engines.get_model_index("Product")
semantic_query_engine = rag_engines.get_semantic_query_engine("Product")  
sql_query_engine = rag_engines.get_sql_query_engine()

In [6]:
print(rag_settings.VECTOR_DB_NAME, rag_settings.VECTOR_DB_TABLE_NAMES)

vector_db ['employeeroles', 'employees', 'producttypes', 'products', 'inventoryitems', 'productinventoryrequirements']


In [7]:
from llama_index.core.tools import QueryEngineTool

vector_tool = QueryEngineTool.from_defaults(
    query_engine=semantic_query_engine,
    description=(
        "Useful for answering semantic questions about coffee shop operations, "
        "including products, employees, inventory, and recipes"
    ),
)


In [8]:
sql_tool = QueryEngineTool.from_defaults(
    query_engine=sql_query_engine,
    description=(
        "Useful for translating a natural language query into a SQL query over "
        "tables containing:EmployeeRole, Employee, ProductType, Product, InventoryItem, ProductInventoryRequirement."
    ),
)

In [9]:
query_engine = rag_patches.MySQLAutoVectorQueryEngine(
    sql_tool,
    vector_tool,
)

In [10]:
response = query_engine.query(
    "how mush is an americano?"
)

INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:llama_index.core.query_engine.sql_join_query_engine:> Querying other query engine: The question 'how mush is an americano?' pertains to coffee shop operations, specifically about products, which is covered by choice (2).


Querying other query engine: The question 'how mush is an americano?' pertains to coffee shop operations, specifically about products, which is covered by choice (2).


INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"


In [11]:
response.response

'Empty Response'

In [12]:
response = query_engine.query(
    "Who is the bartender?"
)

INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:llama_index.core.query_engine.sql_join_query_engine:> Querying SQL database: The question 'Who is the bartender?' is likely to involve querying a database for employee roles, which is covered by the tables mentioned in choice (1), specifically EmployeeRole and Employee.


Querying SQL database: The question 'Who is the bartender?' is likely to involve querying a database for employee roles, which is covered by the tables mentioned in choice (1), specifically EmployeeRole and Employee.


INFO:llama_index.core.indices.struct_store.sql_retriever:> Table desc str: Table 'coffeshop_employeerole' has columns: id (BIGINT), name (VARCHAR(100)), description (TEXT), .

Table 'coffeshop_employee' has columns: id (BIGINT), name (VARCHAR(200)), hire_date (DATE), hourly_rate (NUMERIC(6, 2)), is_active (BOOLEAN), embedding (VECTOR(1024)), role_id (BIGINT),  and foreign keys: ['role_id'] -> coffeshop_employeerole.['id'].

Table 'coffeshop_producttype' has columns: id (BIGINT), name (VARCHAR(100)), description (TEXT), embedding (VECTOR(1024)), .

Table 'coffeshop_product' has columns: id (BIGINT), name (VARCHAR(200)), price (NUMERIC(6, 2)), description (TEXT), is_available (BOOLEAN), embedding (VECTOR(1024)), product_type_id (BIGINT),  and foreign keys: ['product_type_id'] -> coffeshop_producttype.['id'].

Table 'coffeshop_inventoryitem' has columns: id (BIGINT), name (VARCHAR(200)), unit (VARCHAR(50)), quantity (NUMERIC(10, 2)), reorder_level (NUMERIC(10, 2)), cost_per_unit (NUMERIC(

SQL query: SELECT coffeshop_employee.name
FROM coffeshop_employee
JOIN coffeshop_employeerole ON coffeshop_employee.role_id = coffeshop_employeerole.id
WHERE coffeshop_employeerole.name = 'Bartender' AND coffeshop_employee.is_active = TRUE
ORDER BY coffeshop_employee.name;
SQL response: Based on the query results, there is no active bartender currently listed.


INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:llama_index.core.query_engine.sql_join_query_engine:> Transformed query given SQL response: Based on the SQL response, "there is no active bartender currently listed," the original question "Who is the bartender?" has been fully answered. Therefore, the new question is:

None


Transformed query given SQL response: Based on the SQL response, "there is no active bartender currently listed," the original question "Who is the bartender?" has been fully answered. Therefore, the new question is:

None


INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
INFO:llama_index.core.query_engine.sql_join_query_engine:> query engine response: Empty Response


query engine response: Empty Response


INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"


In [13]:
from IPython.display import Markdown, display

display(Markdown(response.response))

Based on the provided information, the response to the original question "Who is the bartender?" is:

**There is no active bartender currently listed.**

This conclusion is drawn from the SQL query response, which indicates that no active bartender is found in the database. The vector store query and response do not add any further information, as the original question has already been fully addressed by the SQL query response.